In [49]:
import os
import psycopg2
from psycopg2.extras import RealDictCursor
from openai import OpenAI

# Load env variables
OPENAI_API_KEY = os.getenv("API_KEY")
BASE_URL = os.getenv("BASE_URL")
client = OpenAI(api_key=OPENAI_API_KEY, base_url = BASE_URL)

# DB and OpenAI config
DB_CONFIG = {
    "host": os.getenv("DB_HOST"),
    "port": int(os.getenv("DB_PORT")),
    "dbname": os.getenv("DB_NAME"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
}

def embed_texts(texts: list[str]) -> list[list[float]]:
    """
    Get embeddings for a batch of texts using OpenAI embeddings API.
    """
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=texts
    )
    return [item.embedding for item in response.data]

from psycopg2.extras import RealDictCursor, execute_values

def update_table_embeddings(table_name: str, id_column: str, column_text: str, batch_size: int = 50):
    """
    Embeds all rows of a given table and text column, storing embeddings
    in the 'embedding' column (float8[]). Uses batch API calls and bulk update.

    Parameters
    ----------
    table_name : str
        Name of the table to embed.
    id_column : str
        Name of the primary key column to identify rows.
    column_text : str
        Name of the text column to embed.
    batch_size : int, optional
        Number of rows to process per batch (default 50).
    """
    with psycopg2.connect(**DB_CONFIG) as conn:
        with conn.cursor(cursor_factory=RealDictCursor) as cur:

            # Ensure embedding column exists
            cur.execute(f"""
                ALTER TABLE {table_name}
                ADD COLUMN IF NOT EXISTS embedding float8[];
            """)

            # Fetch all primary keys and text column
            cur.execute(f"SELECT {id_column}, {column_text} FROM {table_name};")
            rows = cur.fetchall()
            print(f"Total rows to embed from {table_name}: {len(rows)}")

            # Process in batches
            for i in range(0, len(rows), batch_size):
                batch = rows[i:i+batch_size]
                keys = [r[id_column] for r in batch]
                texts = [r[column_text] for r in batch]

                # Get embeddings
                embeddings = embed_texts(texts)  # returns List[List[float]]

                # Create temporary table with same type as original ID
                cur.execute(f"""
                    CREATE TEMP TABLE temp_embeddings (
                        {id_column} {rows[0][id_column].__class__.__name__.lower()} PRIMARY KEY,
                        embedding float8[]
                    );
                """)

                # Bulk insert embeddings
                values = list(zip(keys, embeddings))
                execute_values(cur,
                    f"INSERT INTO temp_embeddings ({id_column}, embedding) VALUES %s",
                    values
                )

                # Update original table (types now match)
                cur.execute(f"""
                    UPDATE {table_name} t
                    SET embedding = tmp.embedding
                    FROM temp_embeddings tmp
                    WHERE t.{id_column} = tmp.{id_column};
                """)

                # Drop temp table for next batch
                cur.execute("DROP TABLE temp_embeddings;")

                print(f"Processed batch {i // batch_size + 1} ({len(batch)} rows)")

        conn.commit()
    print(f"✅ All embeddings updated successfully for table '{table_name}'.")


In [ ]:
update_table_embeddings(table_name= "base_products", id_column = "random_key", column_text = "persian_name", batch_size = 10000)

: 

In [ ]:
import numpy as np
import psycopg2
from psycopg2.extras import RealDictCursor

with psycopg2.connect(**DB_CONFIG) as conn:
    with conn.cursor(cursor_factory=RealDictCursor) as cur:
        cur.execute(f"SELECT {'id'}, {'title'}, embedding FROM {'categories'} WHERE embedding IS NOT NULL;")
        rows = cur.fetchall()

def search_similar_rows_python(table_name, text_column, query, id_column="id", limit=5):
    """
    Fetch embeddings from Postgres and compute cosine similarity in Python.
    """
    query_emb = np.array(embed_text(query), dtype=np.float64)

    # Compute similarity
    results = []
    for row in rows:
        emb = np.array(row["embedding"], dtype=np.float64)
        sim = np.dot(query_emb, emb) / (np.linalg.norm(query_emb) * np.linalg.norm(emb))
        results.append({**row, "similarity": sim})

    # Sort by similarity
    results.sort(key=lambda x: x["similarity"], reverse=True)
    return results[:limit]


In [48]:
results = search_similar_rows_python(
    table_name="categories",
    text_column="title",
    query="لیوان",
    limit=5
)

for r in results:
    print(r)


{'id': 1933, 'title': 'پارچ و لیوان', 'embedding': [-0.02756894938647747, -0.005039037670940161, -0.026257134974002838, -0.049390897154808044, -0.011775106191635132, -0.003071314189583063, 0.013898998498916626, 0.012108266353607178, -0.016845377162098885, -0.0418739877641201, -0.0007164230919443071, -0.006512227468192577, -0.0022110859863460064, 0.015065057203173637, -0.01775115542113781, -0.007787603419274092, 0.028735008090734482, -0.022321688011288643, 0.024986963719129562, 0.0044169663451612, -0.004875060636550188, 0.010671515949070454, -0.04718371480703354, -0.04045805707573891, 0.011858396232128143, 0.03298279270529747, 0.04343567043542862, -0.002699112519621849, 0.018531998619437218, -0.023508567363023758, 0.03875061497092247, -0.02330034412443638, -0.01144194696098566, -0.0014497644733637571, 0.025070253759622574, 0.027985399588942528, 0.012274845503270626, -0.040957797318696976, 0.007147312629967928, 0.010353973135352135, -0.01984381303191185, 0.00454450398683548, -0.030692320

In [22]:
import os
import json
from openai import OpenAI

# Config
OPENAI_API_KEY = os.getenv("API_KEY")
BASE_URL = os.getenv("BASE_URL")
client = OpenAI(api_key=OPENAI_API_KEY, base_url=BASE_URL)

def embed_texts(texts: list[str]) -> list[list[float]]:
    """Get embeddings for a batch of texts using OpenAI embeddings API."""
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=texts,
        dimensions = 256,
    )
    return [item.embedding for item in response.data]

from tqdm import tqdm
def generate_all_embeddings(rows: list[tuple[int, str]], batch_size: int = 1000, save_path: str = "embeddings.jsonl"):
    """
    Generates embeddings for all rows and saves to JSONL for later bulk load.
    
    Parameters
    ----------
    rows : list of tuples
        Each tuple is (id, text)
    batch_size : int
        Number of rows to embed per API call
    save_path : str
        Path to save JSONL file
    """
    with open(save_path, "w", encoding="utf-8") as f:
        for i in tqdm(range(0, len(rows), batch_size)):
            batch = rows[i:i+batch_size]
            ids = [r[0] for r in batch]
            texts = [r[1] for r in batch]
            
            embeddings = embed_texts(texts)

            for _id, emb in zip(ids, embeddings):
                # Save each row as JSON line
                f.write(json.dumps({"id": _id, "embedding": emb}) + "\n")

            # print(f"Processed batch {i // batch_size + 1} ({len(batch)} rows)")

    print(f"✅ All embeddings saved to {save_path}")


In [ ]:
# Suppose you fetched rows from DB
import psycopg2
import os
DB_CONFIG = {
    "host": os.getenv("DB_HOST"),
    "port": int(os.getenv("DB_PORT")),
    "dbname": os.getenv("DB_NAME"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
}

with psycopg2.connect(**DB_CONFIG) as conn:
    with conn.cursor() as cur:
        cur.execute("SELECT random_key, persian_name FROM base_products;")
        rows = cur.fetchall()

In [ ]:
rows = [(row[0],row[1]) for row in rows if row[1] != ""]

In [4]:
rows[0:10]

[('wlfxek', 'تابلو نوری بکلیت مدل طرح بازی گیمنت Gear-5 مدل چراغدار W-s1404'),
 ('yjjgvz', 'داکیماکورا بازی GENSHIN IMPACT - گنشین طرح Scaramouche'),
 ('bcqiyi', 'زودپز روگازی 7 لیتری تلیونیکس مدل TPC3603'),
 ('mrmpti', 'جا ادویه دو طبقه'),
 ('wocxfq',
  'بطری نیم لیتری با درب آرام ریز / بسته ۱۷۰ عددی- پت پلاستیکی ۵۰۰ سی سی طرح الماس (هر عدد ۶۵۰۰تومان)'),
 ('bzqsmx',
  'صندلی کمپ آبی کمپینگ پیک نیک لوکس Silva 3 Camp با فنجان های + میز Mdf تاشو moniev'),
 ('feznnf', 'آفتابه مسافرتی مدل SB12 - کالباسی'),
 ('afhchl', 'پرده شب و روز طوسی-سفید کد RC6-B - ابعاد 160 × 180 سانتی متر'),
 ('acxkxj', 'ساعت دیواری آینه ای مدل پینار - سایز 60 - رنگ برنز بدون گوشواره'),
 ('xdgynu', 'المنت ساندویچ ساز کنوود 1500 وات')]

In [ ]:
# generate_all_embeddings(rows, batch_size=2000, save_path="embeddings.jsonl")

100%|██████████| 512/512 [4:35:02<00:00, 32.23s/it]    


✅ All embeddings saved to embeddings.jsonl


In [9]:
import psycopg2
import psycopg2.extras
import json
import os
from tqdm import tqdm

DB_CONFIG = {
    "host": os.getenv("DB_HOST"),
    "port": int(os.getenv("DB_PORT")),
    "dbname": os.getenv("DB_NAME"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
}

# --- Step 1: Create the table efficiently ---
create_table_sql = """
DROP TABLE IF EXISTS embedding_products;
CREATE TABLE embedding_products (
    random_key TEXT PRIMARY KEY,
    persian_name TEXT,
    embedding FLOAT8[]
);
"""
# --- Step 3: Bulk insert with execute_values ---
def bulk_insert(data, conn):
    insert_sql = """
    INSERT INTO embedding_products (random_key, persian_name, embedding)
    VALUES %s
    """
    with conn.cursor() as cur:
        psycopg2.extras.execute_values(
            cur, insert_sql, data, template=None, page_size=500
        )
    conn.commit()

In [ ]:
import os
import json
import psycopg2
import pandas as pd
from tqdm.notebook import tqdm

DB_CONFIG = {
    "host": os.getenv("DB_HOST"),
    "port": int(os.getenv("DB_PORT")),
    "dbname": os.getenv("DB_NAME"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
}

jsonl_path = "embeddings.jsonl"

# Suppose you already have rows from DB
# id_to_name = dict(rows)  # {random_key: persian_name}

with psycopg2.connect(**DB_CONFIG) as conn:
    with conn.cursor() as cur:
        cur.execute("""
            DROP TABLE IF EXISTS embedding_products;
            CREATE TABLE embedding_products (
                random_key TEXT PRIMARY KEY,
                persian_name TEXT,
                embedding FLOAT8[] CHECK (array_length(embedding, 1) = 256)
            );
        """)
        conn.commit()

    file_size = os.path.getsize(jsonl_path)

    with open(jsonl_path, "r", encoding="utf-8") as f:
        with tqdm(total=file_size, unit="B", unit_scale=True, desc="Loading JSONL into DB") as pbar:
            for chunk in pd.read_json(f, lines=True, chunksize=50000):
                # update progress bar by bytes read so far
                pbar.update(f.tell() - pbar.n)

                # add persian_name from lookup
                # chunk["persian_name"] = chunk["id"].map(id_to_name)

                # convert embeddings list -> Postgres array string
                chunk["embedding"] = chunk["embedding"].apply(
                    lambda x: "{" + ",".join(map(str, x)) + "}"
                )

                # write chunk to DB with COPY
                buf = chunk[["id", "embedding"]].to_csv(
                    index=False, header=False, sep="\t"
                )

                with conn.cursor() as cur:
                    cur.copy_from(
                        io.StringIO(buf),
                        "embedding_products",
                        sep="\t",
                        columns=("random_key", "embedding"),
                    )
                conn.commit()

print("✅ All embeddings streamed into Postgres with progress bar")

